### Test sentiment ML 2

In [1]:
from parse import *
from word_tools import *


In [2]:
from word_tools import make_bagofwords

nb_words = 10000

bow = make_bagofwords("data/corpus.data", 10000)
print(bow[-50:])

['contre', 'être', 'européens', 'ont', 'tous', 'grand', 'même', 'politique', 'bien', 'français', 'président', 'lui', 'ils', 'faire', 'tout', 'comme', 'sont', 'raffarin', 'gilets', 'jaunes', 'renaissance', 'tribune', 'http', 'nous', 'fait', 'mais', 'via', 'européennes', 'france', 'par', 'son', 'plus', 'européenne', 'avec', 'sur', 'aux', 'europe', 'vous', 'dans', 'emmanuel', 'qui', 'pas', 'que', 'une', 'des', 'pour', 'est', 'les', '', 'macron']


In [3]:
from word_tools import vectorize_tweets
vect = vectorize_tweets("data/corpus.data", bow)

In [4]:
import numpy as np
keys = [k for k in vect]

input_len = np.shape(vect[keys[0]]['vectorized'])[0]

print(np.shape(vect[keys[0]]['vectorized']))

(49,)


In [5]:

def extract_data(vectsdict):
    import numpy as np
    vects = []
    labels = []
    
    for key in vectsdict :
        if not np.all(vectsdict[key]['label'] == 0) :
            vects.append(vectsdict[key]['vectorized'])
            labels.append(vectsdict[key]['label'])
    return np.array(vects), np.array(labels)

def decode_output(out_array) :
    if out_array[0] == np.max(out_array) :
        return "irr", np.max(out_array)
    elif out_array[1] == np.max(out_array) :
        return "neg", np.max(out_array)
    elif out_array[2] == np.max(out_array) :
        return "neu",np.max(out_array)
    elif out_array[3] == np.max(out_array) :
        return "pos",np.max(out_array)
    else :
        return "err",0
    
data, labels = extract_data(vect)


percentage_train = 0.8
borne = int(percentage_train*len(data))
X_train = data[:borne]
Y_train = labels[:borne]
X_test = data[borne:]
Y_test = labels[borne:]



In [6]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.embeddings import Embedding

# define the model
model = Sequential()
model.add(Embedding(input_dim=nb_words, output_dim=8, input_length=input_len))
model.add(Flatten())
model.add(Dense(4, activation='sigmoid'))

# compile the model

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# print(model.summary())
# batch = 50
batch = 100
model.fit(X_train, Y_train, epochs=70, batch_size=batch, verbose=1)



Using TensorFlow backend.


Epoch 1/70
3998/3998 [==============================] - 0s 53us/step - loss: 0.6412 - acc: 0.7227
Epoch 2/70
3998/3998 [==============================] - 0s 22us/step - loss: 0.5028 - acc: 0.7706
Epoch 3/70
3998/3998 [==============================] - 0s 19us/step - loss: 0.4562 - acc: 0.7825
Epoch 4/70
3998/3998 [==============================] - 0s 21us/step - loss: 0.4412 - acc: 0.7993
Epoch 5/70
3998/3998 [==============================] - 0s 21us/step - loss: 0.4230 - acc: 0.8165
Epoch 6/70
3998/3998 [==============================] - 0s 21us/step - loss: 0.4024 - acc: 0.8281
Epoch 7/70
3998/3998 [==============================] - 0s 21us/step - loss: 0.3815 - acc: 0.8412
Epoch 8/70
3998/3998 [==============================] - 0s 21us/step - loss: 0.3614 - acc: 0.8542
Epoch 9/70
3998/3998 [==============================] - 0s 21us/step - loss: 0.3426 - acc: 0.8654
Epoch 10/70
3998/3998 [==============================] - 0s 21us/step - loss: 0.3246 - acc: 0.8760
Epoch 11/70
3998/39

In [7]:
loss, accuracy = model.evaluate(X_test, Y_test, verbose=0)
# loss, accuracy = model.evaluate(X_train, Y_train)
print(accuracy)

0.79625


In [9]:
testdata = parse_archive("test.data")

for tw in testdata :
    
    text = processTweet(testdata[tw]['text'])
    
    test_vect = vectorize_tweet(text, "pos", bow, input_len)
#     v = np.array(test_vect['vectorized'])
#     v = v.reshape((1,input_len))

#     res = model.predict([v])[0]
#     out = decode_output(res)
#     print("tweet is evaluated " + out[0])